<a href="https://colab.research.google.com/github/0xZee/financial-agent/blob/main/fin_crewai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CREW AI AGENT : `Financial Reporter`
---

An advanced research assistant by plugging LlamaIndex-powered tools into a CrewAI-powered multi-agent setup.

LlamaIndex is a framework enabling developers to easily build LLM-powered applications over their data; it contains production modules for indexing, retrieval, and prompt/agent orchestration. A core use case is building a generalized QA interface enabling knowledge synthesis over complex questions.

Plugging a LlamaIndex RAG pipeline as a tool into a CrewAI agent setup enables even more sophisticated/advanced research flows

FINANCIAL ANALYST `LANGCHAIN & CREW AI`
---

In [78]:
# Install
!pip install -q 'crewai[tools]' --progress-bar off
!pip install -q langchain-groq --progress-bar off
!pip install -q duckduckgo-search --progress-bar off
!pip install -q yfinance --progress-bar off

In [145]:
#import os
#from google.colab import userdata
#os.environ["GROQ_API"] = userdata.get["GROQ_API"]

In [188]:
from google.colab import userdata
import pandas as pd
import requests as re
from datetime import datetime
import yfinance as yf
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
from langchain.agents import load_tools
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchResults #DuckDuckGoSearchRunTool DuckDuckGoSearchResults DuckDuckGoSearchRun
from langchain_groq import ChatGroq

In [189]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    api_key=userdata.get('GROQ_API')
)

In [190]:
def format_response(response: str) -> str:
    entries = response.split(r"(?<=]), (?=\[)", response)
    return [entry.strip("[]") for entry in entries]

In [170]:
# Search tool
search_tool = DuckDuckGoSearchResults(backend="news", num_results=2 , verbose=True)
response = search_tool.run("PLTR")
print(response)

[snippet: We recently published a list of the 12 Best AI Stocks Leading the 'Big Tech Race' to $4 Trillion According to a Famous Wall Street Analyst. Since Palantir Technologies Inc (NYSE:PLTR) ranks 12th on the list,, title: Is Palantir Technologies Inc (NYSE:PLTR) the Best AI Stock in the $4 Trillion 'AI Race', link: https://www.msn.com/en-us/money/savingandinvesting/is-palantir-technologies-inc-nyse-pltr-the-best-ai-stock-in-the-4-trillion-ai-race/ar-BB1oO0he, date: 2024-06-24T16:01:47+00:00, source: Insider Monkey on MSN.com], [snippet: Shares of data-mining and analytics company Palantir (NYSE:PLTR) fell 5.7% in the morning session after Monness, Crespi, Hardt analyst Brian White downgraded the stock's rating from Neutral to Sell and assigned a $20 price target., title: Why Palantir (PLTR) Shares Are Getting Obliterated Today, link: https://finance.yahoo.com/news/why-palantir-pltr-shares-getting-151958785.html, date: 2024-06-21T15:20:00+00:00, source: YAHOO!Finance][snippet: We re

In [191]:
# Tool Financial data tool
@tool("financial data tool")
def financial_data_tool(ticker):
    """
    Get all the financial data metrics for a given stock ticker name
    """
    # Fetch data for the company
    stock = yf.Ticker(ticker)
    # Initialize the document with the company's ticker
    document = f"Financial Data informations listed by the company with ticker {ticker} :\n ---- \n"
    # Get the info and reco
    document += "\n---\n ## Info and Financial metrics : \n" + str(stock.info)
    document += "\n---\n ## Recommendations : \n" + str(stock.recommendations)
    # Add news to the document
    news = stock.news
    out = "\n---\n ## News for the stock : \n"
    for i in news:
        out += f"# {i['title']} \n Publisher : {i['publisher']} \n Related ticker : {i['relatedTickers']} \n "
    document += out
    # balancesheet
    balance_sheet = pd.DataFrame(stock.balance_sheet)
    document += "\n---\n ## Balance Sheet : \n" + balance_sheet.to_string()
    # cash flow
    cash_flow = pd.DataFrame(stock.cashflow)
    document += "\n---\n ## Cash Flow : \n" + cash_flow.to_string()
    # income statement
    income_statement = pd.DataFrame(stock.income_stmt)
    document += "\n---\n ## Income Statement : \n" + income_statement.to_string()

    return document


In [192]:
# Tool News tool
@tool("search news tool")
def search_news_tool(ticker):
  """
  Get the last news about a given stock ticker name
  """
  return search_tool.run(ticker)


In [193]:
# Tool : Human Tool
humain_tool = load_tools(["human"])

In [194]:
# Define your agents with roles and goals
assistant = Agent(
    role="Customer assistant",
    goal="Identify which {ticker} the customer is interested in",
    backstory="""You a highly Customer assistant helping customer to pick a stock {ticker},
    based on customer interest and domain such ai, cloud, energy.
    """,
    verbose=True,
    memory=True,
    max_iter=5,
    allow_delegation=False,
    tools=humain_tool,
    llm = llm,
)

# financial analyst
analyst = Agent(
    role="Financial Analyst",
    goal="Uncover main financial situation and insights about company {ticker}",
    backstory="""You work at an asset management firm.
  Your goal is to understand tech stock {ticker} based of the financial data.""",
    verbose=True,
    memory=True,
    max_iter=5,
    allow_delegation=False,
    tools=[financial_data_tool],
    llm = llm,
)

researcher = Agent(
    role="News Analyst",
    goal="Uncover news about tech companie {ticker}",
    backstory="""You work at an news office for asset management firm.
  Your goal is to get news of tech stock {ticker} and identify trends.""",
    verbose=True,
    memory=True,
    max_iter=5,
    allow_delegation=False,
    tools=[search_news_tool],
    llm = llm,
)
reporter = Agent(
    role="Financial Reporter",
    goal="Craft compelling content on financial advancements for the stock {ticker}",
    backstory="""You are a renowned Content Strategist for {ticker}, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True,
    memory=True,
    max_iter=5,
    allow_delegation=False,
    llm = llm,
)


In [195]:
# Create tasks for your agents
assistant_task = Task(
    description="""
    Once you have the ticker : stop asking and proceed to the next step.
    Start by asking if the user is interested in a specific stock ticker.
    If not help it by asking a specific sector/industry (eg. ai, cloud, energy) he is intersted in. and provide choice between 5 stocks in this sector.
    """,
    expected_output="{ticker} symbol (eg. : AAPL, IBM, BAC)",
    agent=assistant,
)

analyst_task = Task(
    description=f"""
    Use the financial data tool to get the main financial situation and insights about company ticker.
    the current date is {datetime.now()}
    develop an financial report : present the main data of the Stock with analysis of the price, valuation and performance, then highlights the headwinds that Apple faces.
    Your post should be lowlevel detailed and informative, catering to a professional audience.
    Make it sound professional, provide full analysis.
    """,
    expected_output="Full financial report for {ticker} in markdown format of at least 8 paragraphs",
    agent=assistant,
    context=[assistant_task]
    #output_file='new-blog-post.md',
)

research_task = Task(
    description=f"""
    Use the search news tool to get the last news about company ticker.
    Conduct a Highly detailed analysis of ticker's last news.
    look for the risks and headwinds and identify futur trends.
    the current date is {datetime.now()}
    """,
    expected_output="Full News analysis report for {ticker} with risks, tailwinds, insights and futur trends in bullet points",
    agent=researcher,
    context=[assistant_task],
)

reporter_task = Task(
    description=f"""
    Use the reports provided by the Financial Analyst and the News Analyst to create a financial report.
    Conduct a Highly detailed analysis of ticker's last news.
    look for the risks and headwinds and identify futur trends.
    the current date is {datetime.now()}
    """,
    expected_output="Full Financial report for {ticker} with financial data, risks, tailwinds, insights and futur trends in rich markdown format, Your report should be lowlevel detailed and informative, catering to a professional audience. Make it sound professional, provide full analysis, introduction and conclusion",
    agent=researcher,
    context=[analyst_task, research_task],
)

In [186]:
# the Ai Crew
crew = Crew(
    agents=[assistant, analyst, researcher, reporter],
    tasks=[assistant_task, analyst_task, research_task, reporter_task],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    #llm=llm_groq,
    manager_llm=llm_groq,
    max_iter=15,
    #memory=True,
    #cache=True,
    )


In [187]:
# Run the Crew
result = crew.kickoff()

 [2024-06-25 00:50:07][DEBUG]: == Working Agent: Customer assistant
 [2024-06-25 00:50:07][INFO]: == Starting Task: 
    Start by asking if the user is interested in a specific stock ticker.
    If not help it by asking a specific sector/industry (eg. ai, cloud, energy) he is intersted in. and provide choice between 5 stocks in this sector.
    


> Entering new CrewAgentExecutor chain...
Thought: I need to start by asking the customer if they are interested in a specific stock ticker.

Action: human
Action Input: {"question": "Do you have a specific stock ticker in mind?"}

Do you have a specific stock ticker in mind?
IBM
 

IBM

Thought: The customer has provided a specific stock ticker, IBM.

Action: None needed, I have the final answer.

Final Answer: IBM

> Finished chain.
 [2024-06-25 00:50:17][DEBUG]: == [Customer assistant] Task output: IBM


 [2024-06-25 00:50:17][DEBUG]: == Working Agent: Customer assistant
 [2024-06-25 00:50:17][INFO]: == Starting Task: 
    Use the financia

DATA : YAHOO FINANCE TOOL
---

In [39]:
import yfinance as yf
import pandas as pd

def write_financial_data(ticker):
    # Fetch data for the company
    stock = yf.Ticker(ticker)
    # Initialize the document with the company's ticker
    document = f"Financial Data informations listed by the company with ticker {ticker} :\n ---- \n"
    # Get the info and reco
    document += "\n---\n ## Info and Financial metrics : \n" + str(stock.info)
    document += "\n---\n ## Recommendations : \n" + str(stock.recommendations)
    # Add news to the document
    news = stock.news
    out = "\n---\n ## News for the stock : \n"
    for i in news:
        out += f"# {i['title']} \n Publisher : {i['publisher']} \n Related ticker : {i['relatedTickers']} \n "
    document += out
    # balancesheet
    balance_sheet = pd.DataFrame(stock.balance_sheet)
    document += "\n---\n ## Balance Sheet : \n" + balance_sheet.to_string()
    # cash flow
    cash_flow = pd.DataFrame(stock.cashflow)
    document += "\n---\n ## Cash Flow : \n" + cash_flow.to_string()
    # income statement
    income_statement = pd.DataFrame(stock.income_stmt)
    document += "\n---\n ## Income Statement : \n" + income_statement.to_string()

    # write to file in ./ dir
    with open(f'financial_data_{ticker}.txt', 'w') as file:
        file.write(document)
    print(f'Document loaded and saved : financial_data_{ticker}.txt')

    return document

----

DATA : TAVILY TOOL
---

In [ ]:
# Getting Data from YF and index
GROQ_API = userdata.get('TAVILY_API')

tavily_tool = TavilyToolSpec(api_key=TAVILY_API)
tavily_tool_list = tavily_tool.to_tool_list()
#chat_engine = ReActAgent.from_tools(tools=tavily_tool_list)